In [1]:
#imports

import gym
import time
import numpy as np
import random as rand

Things to do:
1. sarsa from greedy (conditionally done)
2. softmax from epsilon not working maybe because of repeat rewards when you stay on step
3. updating rewards (conditionally done)
4. run traning (run full)
5. egreedy not working (not needed)
6. expected sarsa
7. graphs
8. modify sarsa reward to take reward based on the location arrived and not where it should land (conditionally done)
9. what is temperature? (change factor in the softmax algo)
10. comeback to same spot no reward

most stuff is done just need validation with the plots

In [2]:
class SARSA:
    def __init__(self,size,epsilon,explore):
        self.runs = 10
        self.segments = 500
        self.train_episodes = 10
        self.test_episodes = 1
        self.world_size = size
        self.world = np.zeros((self.world_size,self.world_size,4))
        self.world_usage = np.zeros((self.world_size,self.world_size,4))
        self.current = 0
        self.epsilon = epsilon
        self.action = 0
        
        #environment initialization
        self.env=gym.make("FrozenLake-v1", is_slippery=False)#,render_mode='human')
        self.env.reset()
        self.env.render()
        
        #path
        self.current_path = []

        #exploration number
        self.exploration = explore
        self.decay = epsilon/explore
        self.alpha = 0.1
        self.gamma = 0.1
        self.reward = 0
        self.tmp_next = 0
        
        self.temperature = 1
    
    def modified_start(self):
        self.action = 1
        returnValue=self.env.step(self.action) #take next step
        self.env.render()
        
        self.action = 1
        returnValue=self.env.step(self.action) #take next step
        self.env.render()
        
        self.action = 2
        returnValue=self.env.step(self.action) #take next step
        self.env.render()
        
        self.action = 1
        returnValue=self.env.step(self.action) #take next step
        self.env.render()
        
        #self.action = 2
        #returnValue=self.env.step(self.action) #take next step
        #self.env.render()
        
        self.current = returnValue[0]
    def modified_start1(self):
        self.action = 1
        returnValue=self.env.step(self.action) #take next step
        self.env.render()
        
        self.action = 1
        returnValue=self.env.step(self.action) #take next step
        self.env.render()
        
        self.action = 2
        returnValue=self.env.step(self.action) #take next step
        self.env.render()
        
        self.action = 1
        returnValue=self.env.step(self.action) #take next step
        self.env.render()
        
        self.action = 2
        returnValue=self.env.step(self.action) #take next step
        self.env.render()
        
        self.current = returnValue[0]
    def grid_pos(self,val):
        x = int(val/self.world_size)
        y = val % self.world_size
        return[x,y]
    
    def max_return(self,possiblities):
        top = []
        print("possibilities: ",possiblities)
        current_return = 0
        pos = 0
        for i in possiblities:
            if i > current_return:
                top = [pos]
                current_return = i
            elif i == current_return:
                top.append(pos)
            pos += 1
        #print(top)
        if len(top) > 0:
            random_val = np.random.randint(0,len(top),1)
            return(top[random_val[0]])
        else:
            return(top[0])
    
    def max_reward_return(self,possiblities):
        top = []
        #print("possibilities: ",possiblities)
        current_return = 0
        pos = 0
        for i in possiblities:
            if i > current_return:
                top = [pos]
                current_return = i
            elif i == current_return:
                top.append(pos)
            pos += 1
        #print(top)
        return current_return
    
    def next_space_max_reward(self,movement):
        reward = 0
        if movement == 3:
            pos = self.grid_pos(self.current)
            if pos[0] - 1 < 0:
                reward = 0
            else:
                reward = self.max_reward_return(self.world[pos[0]-1][pos[1]]) #returns action not state
        
        elif movement == 1:
            pos = self.grid_pos(self.current)
            if pos[0] + 1 > 3:
                reward = 0
            else:
                reward = self.max_reward_return(self.world[pos[0]+1][pos[1]]) #returns action not state
        
        elif movement == 0:
            pos = self.grid_pos(self.current)
            if pos[1] - 1 < 0:
                reward = 0
            else:
                reward = self.max_reward_return(self.world[pos[0]][pos[1]-1]) #returns action not state
        
        elif movement == 2:
            pos = self.grid_pos(self.current)
            if pos[1] + 1 > 3:
                reward = 0
            else:
                reward = self.max_reward_return(self.world[pos[0]][pos[1]+1]) #returns action not state
        #print("reward second stage: ",reward)
        return reward
    def next_space_max_reward_flip(self,movement):
        reward = 0
        if movement == 0:
            pos = self.grid_pos(self.current)
            if pos[1] - 1 < 0:
                reward = 0
            else:
                reward = self.max_reward_return(self.world[pos[1]-1][pos[0]]) #returns action not state
        
        elif movement == 2:
            pos = self.grid_pos(self.current)
            if pos[1] + 1 > 3:
                reward = 0
            else:
                reward = self.max_reward_return(self.world[pos[1]+1][pos[0]]) #returns action not state
        
        elif movement == 3:
            pos = self.grid_pos(self.current)
            if pos[0] - 1 < 0:
                reward = 0
            else:
                reward = self.max_reward_return(self.world[pos[1]][pos[0]-1]) #returns action not state
        
        elif movement == 1:
            pos = self.grid_pos(self.current)
            if pos[0] + 1 > 3:
                reward = 0
            else:
                reward = self.max_reward_return(self.world[pos[1]][pos[0]+1]) #returns action not state
        #print("reward second stage: ",reward)
        return reward
    def next_space_max_reward_give_max(self,movement):
        reward = 0
        max_reward = 0
        
        pos = self.grid_pos(self.current)
        #print("position for future max: ",pos)
        if pos[1] - 1 < 0:
            reward = 0
        else:
            reward = self.max_reward_return(self.world[pos[1]-1][pos[0]]) #returns action not state
            #print("reward 1: ",reward)
            if reward > max_reward:
                max_reward = reward


        if pos[1] + 1 > 3:
            reward = 0
        else:
            reward = self.max_reward_return(self.world[pos[1]+1][pos[0]]) #returns action not state
            #print("reward 2: ",reward)
            if reward > max_reward:
                max_reward = reward
        
        if pos[0] - 1 < 0:
            reward = 0
        else:
            reward = self.max_reward_return(self.world[pos[1]][pos[0]-1]) #returns action not state
            #print("reward 3: ",reward)
            if reward > max_reward:
                max_reward = reward
        
        if pos[0] + 1 > 3:
            reward = 0
        else:
            reward = self.max_reward_return(self.world[pos[1]][pos[0]+1]) #returns action not state
            #print("reward 4: ",reward)
            if reward > max_reward:
                max_reward = reward
                
        #print("reward second stage: ",reward)
        return max_reward
    
    def next_space_max_reward_give_max_nonflip(self,movement):
        reward = 0
        max_reward = 0
        
        pos = self.grid_pos(self.current)
        #print("position for future max: ",pos)
        if pos[0] - 1 < 0:
            reward = 0
        else:
            reward = self.max_reward_return(self.world[pos[0]-1][pos[1]]) #returns action not state
            #print("reward 1: ",reward)
            if reward > max_reward:
                max_reward = reward


        if pos[0] + 1 > 3:
            reward = 0
        else:
            reward = self.max_reward_return(self.world[pos[0]+1][pos[1]]) #returns action not state
            #print("reward 2: ",reward)
            if reward > max_reward:
                max_reward = reward
        
        if pos[1] - 1 < 0:
            reward = 0
        else:
            reward = self.max_reward_return(self.world[pos[0]][pos[1]-1]) #returns action not state
            #print("reward 3: ",reward)
            if reward > max_reward:
                max_reward = reward
        
        if pos[1] + 1 > 3:
            reward = 0
        else:
            reward = self.max_reward_return(self.world[pos[0]][pos[1]+1]) #returns action not state
            #print("reward 4: ",reward)
            if reward > max_reward:
                max_reward = reward
                
        #print("reward second stage: ",reward)
        return max_reward
        
    def future_max_return(self,possiblities): #includes future return (TBD)
        top = []
        #print("max: ",possiblities)
        current_return = 0
        pos = 0
        for i in possiblities:
            #print("i",i,pos)
            #new_return = i + self.next_space_max_reward_flip(pos)
            new_return = i
            if new_return > current_return:
            #if i > current_return:
                top = [pos]
                current_return = new_return
            elif new_return == current_return:
                top.append(pos)
            pos += 1
        #print(top)
        if len(top) > 0:
            random_val = np.random.randint(0,len(top),1)
            return(top[random_val[0]])
        else:
            return(top[0])
    
    def E_greedy_action(self):
        random_action = np.random.randint(0,100,2)
        if False: #random_action[0] < self.epsilon:
            action = np.random.randint(0,4,1)[0]
            print("random: ",action)
        else:
            pos = self.grid_pos(self.current)
            print("current position: ",pos)
            action = self.max_return(self.world[pos[0]][pos[1]])
            print("non random: ", action)
        #print("action: ",action)
        return action
    
    def True_greedy_action(self):
        random_action = np.random.randint(0,100,2)
        if random_action[0] < 50:
            action = np.random.randint(0,4,1)[0]
            print("random: ",action)
        else:
            pos = self.grid_pos(self.current)
            print("current position: ",pos)
            action = self.max_return(self.world[pos[0]][pos[1]])
            print("non random: ", action)
        #print("action: ",action)
        return action
    
    def E_greedy_next_step_action(self):
        random_action = np.random.randint(0,100,2)
        pos = self.grid_pos(self.tmp_next)
        print("current position: ",pos)
        action = self.max_return(self.world[pos[0]][pos[1]])
        print("non random: ", action)
        #print("action: ",action)
        return action
    
    def softmax(self):
        pos = self.grid_pos(self.current)
        posible_action = self.world[pos[0]][pos[1]]
        probabilities = np.exp(posible_action/self.temperature)/np.sum(np.exp(posible_action/self.temperature))     
        print("probabilities: ",probabilities)
        
        actions = [0,1,2,3]
        action = rand.choices(actions,weights=probabilities,k=1)[0]
        print("action: ",action)
        return action
    
    def next_softmax(self):
        pos = self.grid_pos(self.tmp_next)
        posible_action = self.world[pos[0]][pos[1]]
        probabilities = np.exp(posible_action/self.temperature)/np.sum(np.exp(posible_action/self.temperature))     
        print("probabilities: ",probabilities)
        
        actions = [0,1,2,3]
        action = rand.choices(actions,weights=probabilities,k=1)[0]
        print("action: ",action)
        return action
    
    def next_expected_softmax(self):
        pos = self.grid_pos(self.tmp_next)
        posible_action = self.world[pos[0]][pos[1]]
        probabilities = np.exp(posible_action/self.temperature)/np.sum(np.exp(posible_action/self.temperature))     
        print("probabilities: ",probabilities)
        expected = probabilities * posible_action
        
        return expected
    
    def SARSA(self): #currently greed based
        action = [0,0]
        random_action = np.random.randint(0,100,2)
        if random_action[0] < self.epsilon:
            action = np.random.randint(0,4,1)[0]
            #print("rand")
        else:
            #print("max")
            pos = self.grid_pos(self.current)
            action = self.future_max_return(self.world[pos[0]][pos[1]])
        
        return action
    
    def SARSA_reward(self):
        quality = 0
        pos = self.grid_pos(self.current)
        print("current pos: ",self.current)
        s_a_reward = self.world[pos[0]][pos[1]][self.action]
        print("current reward: ",s_a_reward)
        print("self.reward: ",self.reward)
        #self.world_usage[pos[0]][pos[1]][self.action] = s_a_reward + self.alpha*(self.reward - s_a_reward + self.gamma * next_space_max_reward(self.action))
        next_reward = self.next_space_max_reward(self.action)
        print("next_reward: ",next_reward)
        quality = s_a_reward + self.alpha*(self.reward - s_a_reward + self.gamma * next_reward)
        print("quality update: ",quality)
        
        return quality
    
    def new_SARSA_reward(self):
        quality = 0
        #estimated reward for this step is based on the estimation from the original position
        pos = self.grid_pos(self.current)
        print("current pos: ",self.current)
        s_a_reward = self.world[pos[0]][pos[1]][self.action]
        
        #next reward is based on the location arrived
        print("current reward: ",s_a_reward)
        print("self.reward: ",self.reward)
        next_action = self.next_softmax()
        pos = self.grid_pos(self.tmp_next)
        next_reward = self.world[pos[0]][pos[1]][next_action]
        print("next_reward: ",next_reward)
        
        quality = s_a_reward + self.alpha*(self.reward - s_a_reward + self.gamma * next_reward)
        print("quality update: ",quality)
        
        return quality
    
    def expected_SARSA_reward(self):
        quality = 0
        #estimated reward for this step is based on the estimation from the original position
        pos = self.grid_pos(self.current)
        print("current pos: ",self.current)
        s_a_reward = self.world[pos[0]][pos[1]][self.action]
        
        #next reward is based on the location arrived
        print("current reward: ",s_a_reward)
        print("self.reward: ",self.reward)
        pos = self.grid_pos(self.tmp_next)
        next_reward = self.next_expected_softmax()
        print("next_reward: ",next_reward)
        
        quality = s_a_reward + self.alpha*(self.reward - s_a_reward + self.gamma * next_reward)
        print("quality update: ",quality)
        
        return quality
    
    def loop(self):
        sucess = []
        reward_log = np.zeros(500)
        for j in range(500):
            for i in range(10):
                self.current = 0
                self.current_path = [self.current]

                self.action = 0
                returnValue=self.env.step(self.action)

                #print('Initial state of the system')
                done = False
                self.env.reset()
                print("+++++++++++++++",i,j,"+++++++++++")
                while not done:

                    self.action = self.softmax() #greedy based action
                    print(self.action)

                    returnValue=self.env.step(self.action) #take next step
                    self.reward=returnValue[1] #get reward
                    self.tmp_next=returnValue[0] #get position
                    print(returnValue)

                    pos = self.grid_pos(self.current)
                    #print("current position: ",pos)
                    if self.tmp_next != self.current:
                        self.world[pos[0]][pos[1]][self.action] = self.new_SARSA_reward() #update world reward
                    #print("world state: ",self.world)

                    self.world_usage[pos[0]][pos[1]][self.action] += 1 #update state action usage count

                    self.current = returnValue[0] #update current location
                    self.current_path.append(self.current) #update path taken
                    done = returnValue[2] #update terminal check
                    #self.env.render()#render new state
                    #time.sleep(1)
                    #print("--------------------------------")
                #print(done)
                reward = returnValue[1]
                if reward > 0:
                    sucess.append(self.current_path)

                #time.sleep(5)

                print(self.current_path)

                self.exploration -= self.decay
                if self.exploration < 0:
                    self.exploration = 0
            
            for i in range(1):
                self.current = 0
                self.current_path = [self.current]

                self.action = 0
                returnValue=self.env.step(self.action)

                print('Initial state of the system')
                done = False
                self.env.reset()
                print("+++++++++++++++ test",j,"+++++++++++")
                while not done:

                    self.action = self.E_greedy_action() #greedy based action
                    print(self.action)

                    returnValue=self.env.step(self.action) #take next step
                    self.reward=returnValue[1] #get reward
                    self.tmp_next=returnValue[0] #get position
                    print(returnValue)

                    pos = self.grid_pos(self.current)
                    print("current position: ",pos)
                    #self.world[pos[0]][pos[1]][self.action] = self.new_SARSA_reward() #update world reward
                    print("world state: ",self.world)

                    #self.world_usage[pos[0]][pos[1]][self.action] += 1 #update state action usage count

                    self.current = returnValue[0] #update current location
                    self.current_path.append(self.current) #update path taken
                    done = returnValue[2] #update terminal check
                    self.env.render()#render new state
                    time.sleep(1)
                    print("--------------------------------")
                print(done)
                reward = returnValue[1]
                if reward > 0:
                    sucess.append(self.current_path)
                if self.current == 15:
                    reward_log[j] = 1

                time.sleep(5)

                print(self.current_path)

                self.exploration -= self.decay
                if self.exploration < 0:
                    self.exploration = 0
            
        self.env.close()
        

In [3]:
sarsa = SARSA(4,0,200)
#sarsa.softmax()
sarsa.loop()

+++++++++++++++ 0 0 +++++++++++
probabilities:  [0.25 0.25 0.25 0.25]
action:  2
2
(1, 0.0, False, {'prob': 1.0})
current pos:  0
current reward:  0.0
self.reward:  0.0
probabilities:  [0.25 0.25 0.25 0.25]
action:  3
next_reward:  0.0
quality update:  0.0
probabilities:  [0.25 0.25 0.25 0.25]
action:  3
3
(1, 0.0, False, {'prob': 1.0})
probabilities:  [0.25 0.25 0.25 0.25]
action:  2
2
(2, 0.0, False, {'prob': 1.0})
current pos:  1
current reward:  0.0
self.reward:  0.0
probabilities:  [0.25 0.25 0.25 0.25]
action:  3
next_reward:  0.0
quality update:  0.0
probabilities:  [0.25 0.25 0.25 0.25]
action:  1
1
(6, 0.0, False, {'prob': 1.0})
current pos:  2
current reward:  0.0
self.reward:  0.0
probabilities:  [0.25 0.25 0.25 0.25]
action:  0
next_reward:  0.0
quality update:  0.0
probabilities:  [0.25 0.25 0.25 0.25]
action:  0
0
(5, 0.0, True, {'prob': 1.0})
current pos:  6
current reward:  0.0
self.reward:  0.0
probabilities:  [0.25 0.25 0.25 0.25]
action:  1
next_reward:  0.0
quality 

--------------------------------
current position:  [0, 0]
possibilities:  [0. 0. 0. 0.]
non random:  3
3
(0, 0.0, False, {'prob': 1.0})
current position:  [0, 0]
world state:  [[[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]]
--------------------------------
current position:  [0, 0]
possibilities:  [0. 0. 0. 0.]
non random:  0
0
(0, 0.0, False, {'prob': 1.0})
current position:  [0, 0]
world state:  [[[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]]
--------------------------------
current position:  [0, 0]
possibilities:  [0. 0. 0. 0.]
non random:  2
2
(1, 0.0, Fal

--------------------------------
current position:  [0, 1]
possibilities:  [0. 0. 0. 0.]
non random:  2
2
(2, 0.0, False, {'prob': 1.0})
current position:  [0, 1]
world state:  [[[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]]
--------------------------------
current position:  [0, 2]
possibilities:  [0. 0. 0. 0.]
non random:  0
0
(1, 0.0, False, {'prob': 1.0})
current position:  [0, 2]
world state:  [[[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]]
--------------------------------
current position:  [0, 1]
possibilities:  [0. 0. 0. 0.]
non random:  3
3
(1, 0.0, Fal

--------------------------------
current position:  [0, 0]
possibilities:  [0. 0. 0. 0.]
non random:  0
0
(0, 0.0, False, {'prob': 1.0})
current position:  [0, 0]
world state:  [[[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]]
--------------------------------
current position:  [0, 0]
possibilities:  [0. 0. 0. 0.]
non random:  1
1
(4, 0.0, False, {'prob': 1.0})
current position:  [0, 0]
world state:  [[[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]]
--------------------------------
current position:  [1, 0]
possibilities:  [0. 0. 0. 0.]
non random:  2
2
(5, 0.0, Tru

--------------------------------
current position:  [0, 0]
possibilities:  [0. 0. 0. 0.]
non random:  1
1
(4, 0.0, False, {'prob': 1.0})
current position:  [0, 0]
world state:  [[[0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]]

 [[0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]]

 [[0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]]

 [[0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.1 0. ]
  [0.  0.  0.  0. ]]]
--------------------------------
current position:  [1, 0]
possibilities:  [0. 0. 0. 0.]
non random:  2
2
(5, 0.0, True, {'prob': 1.0})
current position:  [1, 0]
world state:  [[[0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]]

 [[0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]]

 [[0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]]

 [[0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.1 0. ]
  [0.  0.  0.

...

--------------------------------
current position:  [0, 0]
possibilities:  [0. 0. 0. 0.]
non random:  3
3
(0, 0.0, False, {'prob': 1.0})
current position:  [0, 0]
world state:  [[[0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]]

 [[0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]]

 [[0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]]

 [[0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.1 0. ]
  [0.  0.  0.  0. ]]]
--------------------------------
current position:  [0, 0]
possibilities:  [0. 0. 0. 0.]
non random:  2
2
(1, 0.0, False, {'prob': 1.0})
current position:  [0, 0]
world state:  [[[0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]]

 [[0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]]

 [[0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]]

 [[0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.1 0. ]
  [0.  0.  0

--------------------------------
current position:  [1, 0]
possibilities:  [0. 0. 0. 0.]
non random:  1
1
(8, 0.0, False, {'prob': 1.0})
current position:  [1, 0]
world state:  [[[0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]]

 [[0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]]

 [[0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]]

 [[0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.1 0. ]
  [0.  0.  0.  0. ]]]
--------------------------------
current position:  [2, 0]
possibilities:  [0. 0. 0. 0.]
non random:  2
2
(9, 0.0, False, {'prob': 1.0})
current position:  [2, 0]
world state:  [[[0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]]

 [[0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]]

 [[0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]]

 [[0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.1 0. ]
  [0.  0.  0

--------------------------------
current position:  [0, 0]
possibilities:  [0. 0. 0. 0.]
non random:  0
0
(0, 0.0, False, {'prob': 1.0})
current position:  [0, 0]
world state:  [[[0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]]

 [[0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]]

 [[0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]]

 [[0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.1 0. ]
  [0.  0.  0.  0. ]]]
--------------------------------
current position:  [0, 0]
possibilities:  [0. 0. 0. 0.]
non random:  2
2
(1, 0.0, False, {'prob': 1.0})
current position:  [0, 0]
world state:  [[[0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]]

 [[0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]]

 [[0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]]

 [[0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.1 0. ]
  [0.  0.  0

--------------------------------
current position:  [0, 0]
possibilities:  [0. 0. 0. 0.]
non random:  3
3
(0, 0.0, False, {'prob': 1.0})
current position:  [0, 0]
world state:  [[[0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]]

 [[0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]]

 [[0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]]

 [[0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.1 0. ]
  [0.  0.  0.  0. ]]]
--------------------------------
current position:  [0, 0]
possibilities:  [0. 0. 0. 0.]
non random:  0
0
(0, 0.0, False, {'prob': 1.0})
current position:  [0, 0]
world state:  [[[0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]]

 [[0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]]

 [[0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.  0. ]]

 [[0.  0.  0.  0. ]
  [0.  0.  0.  0. ]
  [0.  0.  0.1 0. ]
  [0.  0.  0

KeyboardInterrupt: 

In [32]:
class SARSA:
    def __init__(self,size,epsilon,explore):
        self.runs = 10
        self.segments = 500
        self.train_episodes = 10
        self.test_episodes = 1
        self.world_size = size
        self.world = np.zeros((self.world_size,self.world_size,4))
        self.world_usage = np.zeros((self.world_size,self.world_size,4))
        self.current = 0
        self.epsilon = epsilon
        self.action = 0
        
        #environment initialization
        self.env=gym.make("FrozenLake-v1", is_slippery=False)#,render_mode='human')
        self.env.reset()
        self.env.render()
        
        #path
        self.current_path = []

        #exploration number
        self.exploration = explore
        self.decay = epsilon/explore
        self.alpha = 0.1
        self.gamma = 0.1
        self.reward = 0
        self.tmp_next = 0
        
        self.temperature = 1
        self.reward_log = np.zeros((10,500,11))
    
    def modified_start(self):
        self.action = 1
        returnValue=self.env.step(self.action) #take next step
        self.env.render()
        
        self.action = 1
        returnValue=self.env.step(self.action) #take next step
        self.env.render()
        
        self.action = 2
        returnValue=self.env.step(self.action) #take next step
        self.env.render()
        
        self.action = 1
        returnValue=self.env.step(self.action) #take next step
        self.env.render()
        
        #self.action = 2
        #returnValue=self.env.step(self.action) #take next step
        #self.env.render()
        
        self.current = returnValue[0]
    def modified_start1(self):
        self.action = 1
        returnValue=self.env.step(self.action) #take next step
        self.env.render()
        
        self.action = 1
        returnValue=self.env.step(self.action) #take next step
        self.env.render()
        
        self.action = 2
        returnValue=self.env.step(self.action) #take next step
        self.env.render()
        
        self.action = 1
        returnValue=self.env.step(self.action) #take next step
        self.env.render()
        
        self.action = 2
        returnValue=self.env.step(self.action) #take next step
        self.env.render()
        
        self.current = returnValue[0]
    def grid_pos(self,val):
        x = int(val/self.world_size)
        y = val % self.world_size
        return[x,y]
    
    def max_return(self,possiblities):
        top = []
        #print("possibilities: ",possiblities)
        current_return = 0
        pos = 0
        for i in possiblities:
            if i > current_return:
                top = [pos]
                current_return = i
            elif i == current_return:
                top.append(pos)
            pos += 1
        #print(top)
        if len(top) > 0:
            random_val = np.random.randint(0,len(top),1)
            return(top[random_val[0]])
        else:
            return(top[0])
    
    def max_reward_return(self,possiblities):
        top = []
        #print("possibilities: ",possiblities)
        current_return = 0
        pos = 0
        for i in possiblities:
            if i > current_return:
                top = [pos]
                current_return = i
            elif i == current_return:
                top.append(pos)
            pos += 1
        #print(top)
        return current_return
    
    def next_space_max_reward(self,movement):
        reward = 0
        if movement == 3:
            pos = self.grid_pos(self.current)
            if pos[0] - 1 < 0:
                reward = 0
            else:
                reward = self.max_reward_return(self.world[pos[0]-1][pos[1]]) #returns action not state
        
        elif movement == 1:
            pos = self.grid_pos(self.current)
            if pos[0] + 1 > 3:
                reward = 0
            else:
                reward = self.max_reward_return(self.world[pos[0]+1][pos[1]]) #returns action not state
        
        elif movement == 0:
            pos = self.grid_pos(self.current)
            if pos[1] - 1 < 0:
                reward = 0
            else:
                reward = self.max_reward_return(self.world[pos[0]][pos[1]-1]) #returns action not state
        
        elif movement == 2:
            pos = self.grid_pos(self.current)
            if pos[1] + 1 > 3:
                reward = 0
            else:
                reward = self.max_reward_return(self.world[pos[0]][pos[1]+1]) #returns action not state
        #print("reward second stage: ",reward)
        return reward
    def next_space_max_reward_flip(self,movement):
        reward = 0
        if movement == 0:
            pos = self.grid_pos(self.current)
            if pos[1] - 1 < 0:
                reward = 0
            else:
                reward = self.max_reward_return(self.world[pos[1]-1][pos[0]]) #returns action not state
        
        elif movement == 2:
            pos = self.grid_pos(self.current)
            if pos[1] + 1 > 3:
                reward = 0
            else:
                reward = self.max_reward_return(self.world[pos[1]+1][pos[0]]) #returns action not state
        
        elif movement == 3:
            pos = self.grid_pos(self.current)
            if pos[0] - 1 < 0:
                reward = 0
            else:
                reward = self.max_reward_return(self.world[pos[1]][pos[0]-1]) #returns action not state
        
        elif movement == 1:
            pos = self.grid_pos(self.current)
            if pos[0] + 1 > 3:
                reward = 0
            else:
                reward = self.max_reward_return(self.world[pos[1]][pos[0]+1]) #returns action not state
        #print("reward second stage: ",reward)
        return reward
    def next_space_max_reward_give_max(self,movement):
        reward = 0
        max_reward = 0
        
        pos = self.grid_pos(self.current)
        #print("position for future max: ",pos)
        if pos[1] - 1 < 0:
            reward = 0
        else:
            reward = self.max_reward_return(self.world[pos[1]-1][pos[0]]) #returns action not state
            #print("reward 1: ",reward)
            if reward > max_reward:
                max_reward = reward


        if pos[1] + 1 > 3:
            reward = 0
        else:
            reward = self.max_reward_return(self.world[pos[1]+1][pos[0]]) #returns action not state
            #print("reward 2: ",reward)
            if reward > max_reward:
                max_reward = reward
        
        if pos[0] - 1 < 0:
            reward = 0
        else:
            reward = self.max_reward_return(self.world[pos[1]][pos[0]-1]) #returns action not state
            #print("reward 3: ",reward)
            if reward > max_reward:
                max_reward = reward
        
        if pos[0] + 1 > 3:
            reward = 0
        else:
            reward = self.max_reward_return(self.world[pos[1]][pos[0]+1]) #returns action not state
            #print("reward 4: ",reward)
            if reward > max_reward:
                max_reward = reward
                
        #print("reward second stage: ",reward)
        return max_reward
    
    def next_space_max_reward_give_max_nonflip(self,movement):
        reward = 0
        max_reward = 0
        
        pos = self.grid_pos(self.current)
        #print("position for future max: ",pos)
        if pos[0] - 1 < 0:
            reward = 0
        else:
            reward = self.max_reward_return(self.world[pos[0]-1][pos[1]]) #returns action not state
            #print("reward 1: ",reward)
            if reward > max_reward:
                max_reward = reward


        if pos[0] + 1 > 3:
            reward = 0
        else:
            reward = self.max_reward_return(self.world[pos[0]+1][pos[1]]) #returns action not state
            #print("reward 2: ",reward)
            if reward > max_reward:
                max_reward = reward
        
        if pos[1] - 1 < 0:
            reward = 0
        else:
            reward = self.max_reward_return(self.world[pos[0]][pos[1]-1]) #returns action not state
            #print("reward 3: ",reward)
            if reward > max_reward:
                max_reward = reward
        
        if pos[1] + 1 > 3:
            reward = 0
        else:
            reward = self.max_reward_return(self.world[pos[0]][pos[1]+1]) #returns action not state
            #print("reward 4: ",reward)
            if reward > max_reward:
                max_reward = reward
                
        #print("reward second stage: ",reward)
        return max_reward
        
    def future_max_return(self,possiblities): #includes future return (TBD)
        top = []
        #print("max: ",possiblities)
        current_return = 0
        pos = 0
        for i in possiblities:
            #print("i",i,pos)
            #new_return = i + self.next_space_max_reward_flip(pos)
            new_return = i
            if new_return > current_return:
            #if i > current_return:
                top = [pos]
                current_return = new_return
            elif new_return == current_return:
                top.append(pos)
            pos += 1
        #print(top)
        if len(top) > 0:
            random_val = np.random.randint(0,len(top),1)
            return(top[random_val[0]])
        else:
            return(top[0])
    
    def E_greedy_action(self):
        random_action = np.random.randint(0,100,2)
        if False: #random_action[0] < self.epsilon:
            action = np.random.randint(0,4,1)[0]
            #print("random: ",action)
        else:
            pos = self.grid_pos(self.current)
            #print("current position: ",pos)
            action = self.max_return(self.world[pos[0]][pos[1]])
            #print("non random: ", action)
        #print("action: ",action)
        return action
    
    def True_greedy_action(self):
        random_action = np.random.randint(0,100,2)
        if random_action[0] < 50:
            action = np.random.randint(0,4,1)[0]
            #print("random: ",action)
        else:
            pos = self.grid_pos(self.current)
            #print("current position: ",pos)
            action = self.max_return(self.world[pos[0]][pos[1]])
            #print("non random: ", action)
        #print("action: ",action)
        return action
    
    def E_greedy_next_step_action(self):
        random_action = np.random.randint(0,100,2)
        pos = self.grid_pos(self.tmp_next)
        #print("current position: ",pos)
        action = self.max_return(self.world[pos[0]][pos[1]])
        #print("non random: ", action)
        #print("action: ",action)
        return action
    
    def softmax(self):
        pos = self.grid_pos(self.current)
        posible_action = self.world[pos[0]][pos[1]]
        probabilities = np.exp(posible_action/self.temperature)/np.sum(np.exp(posible_action/self.temperature))     
        #print("probabilities: ",probabilities)
        
        actions = [0,1,2,3]
        action = rand.choices(actions,weights=probabilities,k=1)[0]
        #print("action: ",action)
        return action
    
    def next_softmax(self):
        pos = self.grid_pos(self.tmp_next)
        posible_action = self.world[pos[0]][pos[1]]
        probabilities = np.exp(posible_action/self.temperature)/np.sum(np.exp(posible_action/self.temperature))     
        #print("probabilities: ",probabilities)
        
        actions = [0,1,2,3]
        action = rand.choices(actions,weights=probabilities,k=1)[0]
        #print("action: ",action)
        return action
    
    def next_expected_softmax(self):
        pos = self.grid_pos(self.tmp_next)
        posible_action = self.world[pos[0]][pos[1]]
        probabilities = np.exp(posible_action/self.temperature)/np.sum(np.exp(posible_action/self.temperature))     
        #print("probabilities: ",probabilities)
        expected = probabilities * posible_action
        
        return expected
    
    def SARSA(self): #currently greed based
        action = [0,0]
        random_action = np.random.randint(0,100,2)
        if random_action[0] < self.epsilon:
            action = np.random.randint(0,4,1)[0]
            #print("rand")
        else:
            #print("max")
            pos = self.grid_pos(self.current)
            action = self.future_max_return(self.world[pos[0]][pos[1]])
        
        return action
    
    def SARSA_reward(self):
        quality = 0
        pos = self.grid_pos(self.current)
        #print("current pos: ",self.current)
        s_a_reward = self.world[pos[0]][pos[1]][self.action]
        #print("current reward: ",s_a_reward)
        #print("self.reward: ",self.reward)
        #self.world_usage[pos[0]][pos[1]][self.action] = s_a_reward + self.alpha*(self.reward - s_a_reward + self.gamma * next_space_max_reward(self.action))
        next_reward = self.next_space_max_reward(self.action)
        #print("next_reward: ",next_reward)
        quality = s_a_reward + self.alpha*(self.reward - s_a_reward + self.gamma * next_reward)
        #print("quality update: ",quality)
        
        return quality
    
    def new_SARSA_reward(self):
        quality = 0
        #estimated reward for this step is based on the estimation from the original position
        pos = self.grid_pos(self.current)
        #print("current pos: ",self.current)
        s_a_reward = self.world[pos[0]][pos[1]][self.action]
        
        #next reward is based on the location arrived
        #print("current reward: ",s_a_reward)
        #print("self.reward: ",self.reward)
        next_action = self.next_softmax()
        pos = self.grid_pos(self.tmp_next)
        next_reward = self.world[pos[0]][pos[1]][next_action]
        #print("next_reward: ",next_reward)
        
        quality = s_a_reward + self.alpha*(self.reward - s_a_reward + self.gamma * next_reward)
        #print("quality update: ",quality)
        
        return quality
    
    def expected_SARSA_reward(self):
        quality = 0
        #estimated reward for this step is based on the estimation from the original position
        pos = self.grid_pos(self.current)
        print("current pos: ",self.current)
        s_a_reward = self.world[pos[0]][pos[1]][self.action]
        
        #next reward is based on the location arrived
        
        #print("current reward: ",s_a_reward)
        #print("self.reward: ",self.reward)
        pos = self.grid_pos(self.tmp_next)
        next_reward = self.next_expected_softmax()
        #print("next_reward: ",next_reward)
        
        quality = s_a_reward + self.alpha*(self.reward - s_a_reward + self.gamma * next_reward)
        #print("quality update: ",quality)
        
        return quality
    
    def loop(self):
        sucess = []
        self.reward_log = np.zeros((10,500,11))
        for run in range(10):
            self.world = np.zeros((self.world_size,self.world_size,4))
            self.world_usage = np.zeros((self.world_size,self.world_size,4))
            for j in range(500):
                for i in range(10):
                    self.current = 0
                    self.current_path = [self.current]

                    self.action = 0
                    returnValue=self.env.step(self.action)

                    #print('Initial state of the system')
                    done = False
                    self.env.reset()
                    print("+++++++++++++++",i,j,"+++++++++++")
                    while not done:

                        self.action = self.softmax() #greedy based action
                        #print(self.action)

                        returnValue=self.env.step(self.action) #take next step
                        self.reward=returnValue[1] #get reward
                        self.reward_log[run][j][i] += self.reward
                        self.tmp_next=returnValue[0] #get position
                        #print(returnValue)

                        pos = self.grid_pos(self.current)
                        #print("current position: ",pos)
                        if self.tmp_next != self.current:
                            self.world[pos[0]][pos[1]][self.action] = self.new_SARSA_reward() #update world reward
                        #print("world state: ",self.world)

                        self.world_usage[pos[0]][pos[1]][self.action] += 1 #update state action usage count

                        self.current = returnValue[0] #update current location
                        self.current_path.append(self.current) #update path taken
                        done = returnValue[2] #update terminal check
                        #self.env.render()#render new state
                        #time.sleep(1)
                        #print("--------------------------------")
                    #print(done)
                    reward = returnValue[1]
                    if reward > 0:
                        sucess.append(self.current_path)

                    #time.sleep(5)

                    print(self.current_path)

                    self.exploration -= self.decay
                    if self.exploration < 0:
                        self.exploration = 0
                print("world state: ",self.world)
                for i in range(1):
                    self.current = 0
                    self.current_path = [self.current]

                    self.action = 0
                    returnValue=self.env.step(self.action)

                    #print('Initial state of the system')
                    done = False
                    self.env.reset()
                    print("+++++++++++++++ test",j,"+++++++++++")
                    while not done:

                        self.action = self.E_greedy_action() #greedy based action
                        #print(self.action)

                        returnValue=self.env.step(self.action) #take next step
                        self.reward=returnValue[1] #get reward
                        self.reward_log[run][j][10] += self.reward
                        self.tmp_next=returnValue[0] #get position
                        #print(returnValue)

                        pos = self.grid_pos(self.current)
                        #print("current position: ",pos)
                        #self.world[pos[0]][pos[1]][self.action] = self.new_SARSA_reward() #update world reward
                        #print("world state: ",self.world)

                        #self.world_usage[pos[0]][pos[1]][self.action] += 1 #update state action usage count

                        self.current = returnValue[0] #update current location
                        self.current_path.append(self.current) #update path taken
                        done = returnValue[2] #update terminal check
                        self.env.render()#render new state
                        #time.sleep(1)
                        #print("--------------------------------")
                    #print(done)
                    reward = returnValue[1]
                    if reward > 0:
                        sucess.append(self.current_path)


                    #time.sleep(5)

                    print(self.current_path)

                    self.exploration -= self.decay
                    if self.exploration < 0:
                        self.exploration = 0

            #self.env.close()


In [ ]:
sarsa = SARSA(4,0,200)
#sarsa.softmax()
sarsa.loop()

+++++++++++++++ 0 0 +++++++++++
[0, 4, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 3, 2, 1, 0, 0, 1, 0, 1, 0, 0, 0, 4, 8, 4, 8, 4, 5]
+++++++++++++++ 1 0 +++++++++++
[0, 4, 0, 0, 0, 4, 5]
+++++++++++++++ 2 0 +++++++++++
[0, 4, 0, 4, 4, 8, 4, 8, 8, 12]
+++++++++++++++ 3 0 +++++++++++
[0, 1, 1, 1, 2, 2, 2, 2, 1, 2, 2, 3, 3, 3, 2, 1, 1, 1, 1, 2, 1, 5]
+++++++++++++++ 4 0 +++++++++++
[0, 1, 1, 0, 4, 8, 9, 8, 12]
+++++++++++++++ 5 0 +++++++++++
[0, 4, 4, 5]
+++++++++++++++ 6 0 +++++++++++
[0, 4, 0, 4, 0, 0, 1, 5]
+++++++++++++++ 7 0 +++++++++++
[0, 1, 0, 4, 8, 9, 8, 12]
+++++++++++++++ 8 0 +++++++++++
[0, 0, 1, 2, 6, 5]
+++++++++++++++ 9 0 +++++++++++
[0, 0, 0, 4, 8, 4, 4, 5]
world state:  [[[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]]
+++++++++++++++ test 0 +++++++++++
[0, 0, 0, 4, 5]
+++++

[0, 1, 0, 1, 1, 1, 1, 0, 1, 2, 6, 5]
+++++++++++++++ 0 9 +++++++++++
[0, 1, 2, 6, 5]
+++++++++++++++ 1 9 +++++++++++
[0, 0, 0, 1, 1, 0, 4, 8, 8, 9, 5]
+++++++++++++++ 2 9 +++++++++++
[0, 1, 1, 5]
+++++++++++++++ 3 9 +++++++++++
[0, 0, 4, 4, 8, 8, 9, 10, 6, 10, 14, 14, 10, 6, 7]
+++++++++++++++ 4 9 +++++++++++
[0, 1, 0, 0, 4, 5]
+++++++++++++++ 5 9 +++++++++++
[0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 2, 2, 6, 10, 14, 13, 13, 12]
+++++++++++++++ 6 9 +++++++++++
[0, 4, 0, 0, 0, 1, 1, 2, 3, 3, 3, 3, 2, 6, 10, 6, 7]
+++++++++++++++ 7 9 +++++++++++
[0, 0, 0, 1, 2, 1, 5]
+++++++++++++++ 8 9 +++++++++++
[0, 4, 5]
+++++++++++++++ 9 9 +++++++++++
[0, 0, 1, 2, 1, 5]
world state:  [[[0.0e+00 0.0e+00 0.0e+00 0.0e+00]
  [0.0e+00 0.0e+00 0.0e+00 0.0e+00]
  [0.0e+00 0.0e+00 0.0e+00 0.0e+00]
  [0.0e+00 0.0e+00 0.0e+00 0.0e+00]]

 [[0.0e+00 0.0e+00 0.0e+00 0.0e+00]
  [0.0e+00 0.0e+00 0.0e+00 0.0e+00]
  [0.0e+00 1.9e-05 0.0e+00 0.0e+00]
  [0.0e+00 0.0e+00 0.0e+00 0.0e+00]]

 [[0.0e+00 0.0e+00 0.0e+00 0.0e+00]
  [0

[0, 0, 4, 0, 0, 4, 8, 8, 8, 9, 10, 14, 15]
+++++++++++++++ 0 15 +++++++++++
[0, 0, 4, 4, 8, 8, 8, 9, 10, 9, 5]
+++++++++++++++ 1 15 +++++++++++
[0, 0, 0, 1, 5]
+++++++++++++++ 2 15 +++++++++++
[0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 4, 0, 1, 0, 0, 1, 5]
+++++++++++++++ 3 15 +++++++++++
[0, 4, 0, 0, 0, 0, 4, 4, 5]
+++++++++++++++ 4 15 +++++++++++
[0, 4, 4, 4, 4, 4, 5]
+++++++++++++++ 5 15 +++++++++++
[0, 0, 1, 1, 2, 1, 1, 5]
+++++++++++++++ 6 15 +++++++++++
[0, 1, 5]
+++++++++++++++ 7 15 +++++++++++
[0, 0, 4, 4, 0, 4, 5]
+++++++++++++++ 8 15 +++++++++++
[0, 4, 4, 4, 5]
+++++++++++++++ 9 15 +++++++++++
[0, 4, 0, 1, 2, 2, 2, 2, 3, 2, 3, 3, 7]
world state:  [[[0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]
  [0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]
  [0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]
  [0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]]

 [[0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]
  [0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]
  [0.0000e+00 1.9000e-05 0.0000e+00 0.0000e+00]
  [0.0

[0, 4, 8, 9, 10, 14, 15]
+++++++++++++++ 0 21 +++++++++++
[0, 4, 5]
+++++++++++++++ 1 21 +++++++++++
[0, 4, 4, 4, 0, 4, 8, 4, 0, 0, 0, 1, 1, 5]
+++++++++++++++ 2 21 +++++++++++
[0, 0, 1, 0, 4, 0, 1, 5]
+++++++++++++++ 3 21 +++++++++++
[0, 4, 0, 0, 0, 4, 0, 1, 1, 1, 0, 1, 2, 2, 2, 1, 5]
+++++++++++++++ 4 21 +++++++++++
[0, 1, 0, 0, 0, 0, 4, 4, 5]
+++++++++++++++ 5 21 +++++++++++
[0, 0, 0, 0, 0, 0, 1, 1, 0, 4, 8, 4, 5]
+++++++++++++++ 6 21 +++++++++++
[0, 0, 1, 0, 0, 1, 1, 2, 1, 0, 0, 4, 4, 8, 4, 8, 8, 9, 8, 8, 12]
+++++++++++++++ 7 21 +++++++++++
[0, 1, 2, 3, 7]
+++++++++++++++ 8 21 +++++++++++
[0, 0, 4, 8, 9, 13, 12]
+++++++++++++++ 9 21 +++++++++++
[0, 4, 5]
world state:  [[[0.00000000e+00 3.71079124e-11 6.78414904e-15 0.00000000e+00]
  [2.33968541e-13 0.00000000e+00 1.71000000e-09 0.00000000e+00]
  [0.00000000e+00 1.90000000e-07 0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]]

 [[0.00000000e+00 4.45051901e-09 0.00000000e+00 2.53968597e-

[0, 4, 8, 9, 10, 14, 15]
+++++++++++++++ 0 26 +++++++++++
[0, 4, 5]
+++++++++++++++ 1 26 +++++++++++
[0, 1, 0, 0, 0, 0, 4, 5]
+++++++++++++++ 2 26 +++++++++++
[0, 0, 0, 0, 4, 0, 4, 8, 9, 13, 12]
+++++++++++++++ 3 26 +++++++++++
[0, 0, 0, 0, 4, 5]
+++++++++++++++ 4 26 +++++++++++
[0, 1, 2, 2, 1, 0, 4, 5]
+++++++++++++++ 5 26 +++++++++++
[0, 0, 1, 5]
+++++++++++++++ 6 26 +++++++++++
[0, 4, 4, 4, 0, 4, 0, 4, 8, 12]
+++++++++++++++ 7 26 +++++++++++
[0, 4, 0, 0, 0, 0, 0, 4, 4, 0, 4, 8, 4, 8, 8, 12]
+++++++++++++++ 8 26 +++++++++++
[0, 0, 4, 5]
+++++++++++++++ 9 26 +++++++++++
[0, 0, 0, 4, 0, 4, 4, 5]
world state:  [[[0.00000000e+00 3.81787261e-10 1.10349000e-10 0.00000000e+00]
  [1.47287722e-12 0.00000000e+00 5.28351141e-09 0.00000000e+00]
  [2.67607118e-11 3.27742363e-07 0.00000000e+00 0.00000000e+00]
  [1.90000000e-09 0.00000000e+00 0.00000000e+00 0.00000000e+00]]

 [[0.00000000e+00 1.55628118e-08 0.00000000e+00 5.61890950e-12]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+0

...

[0, 4, 8, 9, 13, 14, 15]
+++++++++++++++ 0 374 +++++++++++
[0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 5]
+++++++++++++++ 1 374 +++++++++++
[0, 0, 1, 1, 0, 1, 1, 0, 4, 5]
+++++++++++++++ 2 374 +++++++++++
[0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 2, 3, 3, 3, 2, 2, 6, 2, 3, 3, 2, 6, 5]
+++++++++++++++ 3 374 +++++++++++
[0, 1, 0, 0, 0, 0, 0, 4, 4, 8, 4, 8, 4, 0, 0, 0, 0, 0, 4, 0, 1, 2, 6, 2, 6, 2, 3, 3, 3, 3, 7]
+++++++++++++++ 4 374 +++++++++++
[0, 4, 4, 0, 4, 8, 4, 0, 0, 0, 4, 8, 8, 8, 4, 8, 12]
+++++++++++++++ 5 374 +++++++++++
[0, 0, 1, 2, 2, 2, 2, 2, 6, 5]
+++++++++++++++ 6 374 +++++++++++
[0, 0, 0, 0, 1, 1, 5]
+++++++++++++++ 7 374 +++++++++++
[0, 0, 1, 5]
+++++++++++++++ 8 374 +++++++++++
[0, 0, 1, 0, 1, 0, 0, 0, 0, 4, 8, 9, 5]
+++++++++++++++ 9 374 +++++++++++
[0, 1, 2, 3, 7]
world state:  [[[0.00000000e+00 3.80659995e-08 1.96726030e-08 0.00000000e+00]
  [7.83234085e-10 0.00000000e+00 7.52792234e-07 0.00000000e+00]
  [2.13131973e-08 2.92959544e-05 1.77637856e-08 0.00000000e+00]
  [9.54782492e-07 0.00

[0, 4, 8, 9, 13, 14, 15]
+++++++++++++++ 0 379 +++++++++++
[0, 0, 0, 1, 0, 0, 4, 8, 12]
+++++++++++++++ 1 379 +++++++++++
[0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 8, 8, 8, 12]
+++++++++++++++ 2 379 +++++++++++
[0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 5]
+++++++++++++++ 3 379 +++++++++++
[0, 0, 0, 4, 4, 4, 8, 8, 9, 10, 6, 7]
+++++++++++++++ 4 379 +++++++++++
[0, 4, 8, 8, 9, 5]
+++++++++++++++ 5 379 +++++++++++
[0, 0, 0, 0, 1, 0, 0, 1, 2, 6, 2, 1, 5]
+++++++++++++++ 6 379 +++++++++++
[0, 4, 5]
+++++++++++++++ 7 379 +++++++++++
[0, 0, 0, 0, 4, 0, 4, 5]
+++++++++++++++ 8 379 +++++++++++
[0, 1, 2, 6, 2, 6, 2, 2, 2, 6, 5]
+++++++++++++++ 9 379 +++++++++++
[0, 1, 5]
world state:  [[[0.00000000e+00 6.28892806e-09 7.45399128e-09 0.00000000e+00]
  [5.40432905e-10 0.00000000e+00 6.59593183e-07 0.00000000e+00]
  [2.06735074e-08 3.53721504e-05 1.86151841e-08 0.00000000e+00]
  [1.15515871e-06 0.00000000e+00 0.00000000e+00 0.00000000e+00]]

 [[0.00000000e+00 4.56025458e-07 0.00000000e+00 8.99301017e-10]
  [0.00

[0, 1, 2, 6, 10, 14, 15]
+++++++++++++++ 0 384 +++++++++++
[0, 4, 0, 1, 5]
+++++++++++++++ 1 384 +++++++++++
[0, 1, 1, 0, 4, 8, 9, 8, 8, 8, 9, 5]
+++++++++++++++ 2 384 +++++++++++
[0, 0, 1, 5]
+++++++++++++++ 3 384 +++++++++++
[0, 4, 8, 12]
+++++++++++++++ 4 384 +++++++++++
[0, 1, 0, 1, 5]
+++++++++++++++ 5 384 +++++++++++
[0, 0, 1, 0, 1, 0, 4, 4, 4, 0, 0, 0, 0, 4, 0, 1, 5]
+++++++++++++++ 6 384 +++++++++++
[0, 0, 1, 1, 0, 0, 0, 4, 5]
+++++++++++++++ 7 384 +++++++++++
[0, 0, 1, 5]
+++++++++++++++ 8 384 +++++++++++
[0, 1, 2, 2, 6, 10, 11]
+++++++++++++++ 9 384 +++++++++++
[0, 1, 1, 0, 4, 0, 1, 2, 2, 1, 5]
world state:  [[[0.00000000e+00 1.52196202e-08 1.25525648e-08 0.00000000e+00]
  [1.19465103e-09 0.00000000e+00 4.33306026e-07 0.00000000e+00]
  [2.20901064e-08 3.18349354e-05 1.67536657e-08 0.00000000e+00]
  [1.15515871e-06 0.00000000e+00 0.00000000e+00 0.00000000e+00]]

 [[0.00000000e+00 1.22166566e-06 0.00000000e+00 1.01564683e-09]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00

[0, 1, 2, 6, 10, 14, 15]
+++++++++++++++ 0 389 +++++++++++
[0, 1, 1, 0, 0, 0, 4, 5]
+++++++++++++++ 1 389 +++++++++++
[0, 1, 1, 2, 6, 5]
+++++++++++++++ 2 389 +++++++++++
[0, 0, 0, 4, 5]
+++++++++++++++ 3 389 +++++++++++
[0, 0, 1, 5]
+++++++++++++++ 4 389 +++++++++++
[0, 0, 4, 0, 4, 5]
+++++++++++++++ 5 389 +++++++++++
[0, 4, 0, 0, 0, 4, 4, 8, 12]
+++++++++++++++ 6 389 +++++++++++
[0, 0, 4, 4, 8, 8, 8, 9, 10, 9, 10, 9, 13, 13, 14, 13, 14, 10, 6, 5]
+++++++++++++++ 7 389 +++++++++++
[0, 1, 0, 0, 4, 0, 0, 1, 5]
+++++++++++++++ 8 389 +++++++++++
[0, 0, 0, 1, 1, 0, 4, 5]
+++++++++++++++ 9 389 +++++++++++
[0, 0, 0, 4, 4, 8, 8, 12]
world state:  [[[0.00000000e+00 5.33973037e-09 8.80826574e-09 0.00000000e+00]
  [6.78855536e-10 0.00000000e+00 8.56550995e-07 0.00000000e+00]
  [2.15345819e-08 2.82206365e-05 2.13885084e-08 0.00000000e+00]
  [1.46976990e-06 0.00000000e+00 0.00000000e+00 0.00000000e+00]]

 [[0.00000000e+00 7.33137894e-07 0.00000000e+00 8.67639481e-10]
  [0.00000000e+00 0.00000000e+

[0, 4, 8, 9, 13, 14, 15]
+++++++++++++++ 0 394 +++++++++++
[0, 1, 0, 0, 1, 0, 1, 2, 6, 10, 14, 14, 15]
+++++++++++++++ 1 394 +++++++++++
[0, 1, 2, 2, 2, 3, 3, 3, 3, 7]
+++++++++++++++ 2 394 +++++++++++
[0, 0, 0, 0, 4, 0, 1, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4, 8, 12]
+++++++++++++++ 3 394 +++++++++++
[0, 0, 4, 8, 4, 0, 0, 1, 5]
+++++++++++++++ 4 394 +++++++++++
[0, 4, 0, 1, 0, 1, 5]
+++++++++++++++ 5 394 +++++++++++
[0, 4, 0, 4, 5]
+++++++++++++++ 6 394 +++++++++++
[0, 4, 8, 9, 13, 12]
+++++++++++++++ 7 394 +++++++++++
[0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 4, 4, 4, 0, 4, 4, 5]
+++++++++++++++ 8 394 +++++++++++
[0, 1, 5]
+++++++++++++++ 9 394 +++++++++++
[0, 1, 5]
world state:  [[[0.00000000e+00 1.76455028e-08 1.18935502e-08 0.00000000e+00]
  [9.29604047e-10 0.00000000e+00 3.71545807e-07 0.00000000e+00]
  [1.14577640e-08 1.60488130e-05 2.47476229e-08 0.00000000e+00]
  [1.39651831e-06 0.00000000e+00 0.00000000e+00 0.00000000e+00]]

 [[0.00000000e+00 3.56386802e-07 0.00000000e+00 1.07179289e-0

In [17]:
sarsa.reward_log[0][40]

array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1.])